In [1]:
import sys
sys.path.append("./../../src")
import pandas as pd
import numpy as np

import torch
from torch import nn, optim

In [2]:
from vae_with_dataloader import VAE, VAETrainerWithDataLoader
from dataloader import PatientEigenVectorDataset
from visualizer_helper import Visualizer

import pickle

from spectral_config_reader import SpectralConfig
import os
import math
from sklearn.model_selection import train_test_split

In [3]:
###Load Configuration file
config = SpectralConfig('./spectral_config.ini')
experiment_name=config.experiment_name
config.__dict__

{'patient_evector_path': './data/L_evecs100_K50.p',
 'experiment_name': '200318_spectral_K50_evec100',
 'encoder_dim': [(250, 500), (500, 250), (250, 100)],
 'latent_dim': 25,
 'decoder_dim': [(250, 500), (500, 250), (250, 100)],
 'use_relu_encoder': True,
 'use_relu_decoder': True,
 'kld_beta': 1.0}

In [4]:
###Load Data
evec_dataset = PatientEigenVectorDataset(eigenvector_path=config.patient_evector_path)

Loaded eigenvector matrix of size (42517, 100)


In [5]:
# icd9codes = pd.read_csv(config.icd9codes_path)
# icd_analyzer = ICDAnalysisHelper(icd9codes_df = icd9codes, patient_icd_df = patient_icd_df)
#icd_analyzer.lookup_icds(icd9codes, ["4019", "41401"])
visualizer = Visualizer()
patient_icd_dataset = evec_dataset

In [6]:
###Load Model
model = VAE(
    feature_dim = patient_icd_dataset.get_feat_dim(), 
    encoder_dim = config.encoder_dim,
    latent_dim = config.latent_dim,
    decoder_dim = config.decoder_dim,
    use_relu_encoder= config.use_relu_encoder,
    use_relu_decoder= config.use_relu_decoder
)

model = model.double()

optimizer = optim.Adam(model.parameters(), lr=0.005)

###Load Trainer
trainer = VAETrainerWithDataLoader(
    model=model, 
    optimizer=optimizer,
    experiment_name=config.experiment_name,
    kld_beta=config.kld_beta
)

In [7]:
training_params = {'batch_size':32, 'shuffle': True}
training_generator = torch.utils.data.DataLoader(patient_icd_dataset, **training_params)

In [ ]:
###Train Model
trainer.train(
    training_generator=training_generator,
    epochs=80,
    save_model_interval=2,
    clip_gradients=False
)

In [ ]:
trainer.plot_elbo()
trainer.plot_bce()
trainer.plot_kld()

In [ ]:
###Load pre-trained model
epoch = 6
model.load_state_dict(torch.load("./VAE_exp_{}_epoch_{}.pkl".format(experiment_name, epoch)))
trainer.model = model

In [ ]:
###Encode data
encoding_params = {'batch_size':64, 'shuffle': False}
encoding_generator = torch.utils.data.DataLoader(patient_icd_dataset, **encoding_params)

latent, means, var = trainer.encode_data(encoding_generator)
pickle.dump(latent, open("latent_epochs_{}.p".format(epoch), 'wb'))
pickle.dump(means, open("means_epochs_{}.p".format(epoch), 'wb'))
pickle.dump(var, open("vars_epochs_{}.p".format(epoch), 'wb'))

In [ ]:
###Get UMAP representations
X_umap = visualizer.umap_embedding(latent, n_components=3)
mean_umap = visualizer.umap_embedding(means, n_components=3)
var_umap = visualizer.umap_embedding(var, n_components=3)

pickle.dump(X_umap, open("X_umap_3d.p", 'wb'))
pickle.dump(mean_umap, open("mean_umap_3d.p", 'wb'))
pickle.dump(var_umap, open("var_umap_3d.p", 'wb'))